In [4]:
%%timeit -r 10 -n 30

import psycopg2

conn = psycopg2.connect(dbname="database4test", user="postgres", password="oirad1993")
cur = conn.cursor()

query = """
SELECT ae_inner."nomemodello"
FROM "apparecchiatureelettroniche" ae_inner
WHERE (
    SELECT digest(CONCAT(ae_inner."manualeutente", ae_inner."nomemodello"), 'sha1')
    ) = (
    SELECT digest(CONCAT(ae."manualeutente", ae."nomemodello"), 'sha1')
    FROM "apparecchiatureelettroniche" ae
    WHERE ae."id" = ae_inner."id"
    )
    AND LENGTH(ae_inner."manualeutente") > 0
    AND ae_inner."id" IN (
        SELECT po."id"
        FROM "prodottiordinati" po
        WHERE po."numeroordine" IN (
            SELECT oc."numeroordine"
            FROM "ordiniclienti" oc
            WHERE oc."cittaspedizione" = 'Milano'
                AND oc."dataordine" BETWEEN '2023-01-01' AND '2023-07-31'
        )
    );

"""

cur.execute(query)
results = cur.fetchall()

for row in results:
    print(row[0])

conn.close()


104 ms ± 9.15 ms per loop (mean ± std. dev. of 10 runs, 30 loops each)
